In [3]:
#!pip install tabpy_client

In [6]:
import snowflake.connector
import pandas as pd
import seaborn as sns
import plotly.express as px
import warnings
import numpy as np
import matplotlib.pyplot as plt
import os
import tabpy_client
# usr = os.environ['svc_user']
# pwd = os.environ['svc_pwd']

usr = 'SVC_PROMO'
pwd = 'nBrVtHzLh8dk2Kvv'

warnings.filterwarnings('ignore')

In [7]:
def sf_connect():
    con = snowflake.connector.connect(  
        account = 'chewy.us-east-1',
        user = usr,
        password=pwd,
        autocommit = False,
        database= 'EDLDB',
        warehouse = 'IT_WH') #EDLDB_DSE_DEV
    return con

In [8]:
def exec_qry_all(qry):
    con = sf_connect()
    cur = con.cursor()
    try:
        cur.execute(qry,timeout=3600)
        return cur.fetch_pandas_all()   
    except Exception as e:
        print(e)
    finally:
        cur.close()

In [11]:
qry = '''SELECT * FROM EDLDB.ECOM_SANDBOX.CTR_TEMP_SV_ZONE_ALL'''
dataset = exec_qry_all(qry)
dataset.head()

,SESSIONS_DATE_EST,CHEWY_FINANCIAL_REPORTING_PERIOD,DEVICE_CATEGORY,CMS_LEVEL,PLACEMENT_SLOT_UID,WIDGET_POSITION,WIDGET_TYPE,WIDGET_TYPE_VALUE,ZONE,WIDGET_NAME,...,ASSET_SEGMENT_VIEWS,WIDGET_SEGMENT_CLICKS,ASSET_SEGMENT_CLICKS,PAGE_IMPRESSIONS,ATTRIBUTED_REVENUE,ATTRIBUTED_UNITS,ATTRIBUTED_ORDER,AUTOSHIP_ATTRIBUTED_REVENUE,AUTOSHIP_ATTRIBUTED_UNITS,AUTOSHIP_ATTRIBUTED_ORDER
0,2022-04-08,2022-P03,mobile,asset,33038980881509905,30,grid,circle-tiles,None,nexgard-body-30-grid,...,1.0,NaN,1.0,NaN,91.986667,1.333333,0.333333,91.986667,1.333333,0.333333
1,2022-04-14,2022-P03,mobile,asset,None,None,nav-flyout,nav-flyout-brand-tiles,None,dog-popular-brands-brand,...,1.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2022-04-10,2022-P03,mobile,asset,14-20,None,standard-sku-collection,None,None,cat-young-body-20-promo-standard-sku-collection,...,1.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2022-03-27,2022-P02,desktop,asset,None,None,nav-flyout,nav-flyout-brand-tiles,None,fish-popular-brands-brand,...,1.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2022-04-11,2022-P03,desktop,asset,5158508964836753,30,grid,circle-tiles,None,chewy-picks-body-30-grid,...,1.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
# def asset_reordering(widget_name, start_date = '2022-04-07', end_date = '2022-04-21'):
#     #Filter by widget, date range and calculate CTR, UNIT/K
#     df = dataset[(dataset['WIDGET_NAME'] == widget_name)&(dataset['CMS_LEVEL'] == 'asset')]
#     df['SESSIONS_DATE_EST'] = df['SESSIONS_DATE_EST'].astype(str)
#     df = df[(df['SESSIONS_DATE_EST'] >= start_date)&(df['SESSIONS_DATE_EST'] <= end_date)]
#     #print("Date Range: {} - {}".format(df['SESSIONS_DATE_EST'].min(),df['SESSIONS_DATE_EST'].max()))

#     df['CTR'] = (df['ASSET_SEGMENT_CLICKS']/df['ASSET_SEGMENT_VIEWS']*100).round(2)
#     df['CTR'].replace([np.inf, -np.inf], np.nan, inplace=True)
#     df['CTR'] = df['CTR'].fillna(0)

#     df['UNITS/K'] = ((df['ATTRIBUTED_UNITS']/df['ASSET_SEGMENT_VIEWS'])*1000).round(2)
#     df['UNITS/K'].replace([np.inf, -np.inf], np.nan, inplace=True)
#     df['UNITS/K'] = df['UNITS/K'].fillna(0)

#     #### NEW ####
#     df['REV/K'] = ((df['ATTRIBUTED_REVENUE']/df['ASSET_SEGMENT_VIEWS'])*1000).round(2)
#     df['REV/K'].replace([np.inf, -np.inf], np.nan, inplace=True)
#     df['REV/K'] = df['REV/K'].fillna(0)
    
#     #Calculate mean impressions by assets
#     asset_impressions = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].sum())
#     print(asset_impressions)
#     # asset_impressions = [x / sum(asset_impressions) for x in asset_impressions] #Calculate probability score of impressions by assets
#     asset_impressions = [x / df['ASSET_SEGMENT_VIEWS'].sum() for x in asset_impressions] #Calculate probability score of impressions by assets
#     asset_names = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean().index)
#     asset_dict = dict(zip(asset_names,asset_impressions))

#     #Create probability column to sample assets by probability score
#     df['PROB'] = df['CREATIVE_NAME'].map(asset_dict)
    
#     #Resample by CTR and UNIT/K
#     df_ctr = df[['CREATIVE_NAME','PROB','CTR']]
#     resamples_ctr = []
#     for i in range(1000):
#         a = df_ctr.sample(n=5000, weights='PROB', replace = True)
#         b = df[['CREATIVE_NAME','PROB','CTR']]
#         resamples_ctr.append(pd.concat([a,b]))
#     #resamples_ctr = [df_ctr.sample(n=5000, weights='PROB', replace = True) for i in range(1000)]
#     sol_ctr = []

#     df_units = df[['CREATIVE_NAME','PROB','UNITS/K']]
#     resamples_units = []
#     for i in range(1000):
#         a = df_units.sample(n=5000, weights='PROB', replace = True)
#         b = df[['CREATIVE_NAME','PROB','UNITS/K']]
#         resamples_units.append(pd.concat([a,b]))   
#     #resamples_units = [df_units.sample(n=5000, weights='PROB', replace = True) for i in range(1000)]
#     sol_units = []

#     #### NEW ####
#     df_rev = df[['CREATIVE_NAME','PROB','REV/K']]
#     resamples_rev = []
#     #resamples_rev = [df_rev.sample(n=5000, weights='PROB', replace = True) for i in range(1000)]
#     for i in range(1000):
#         a = df_rev.sample(n=5000, weights='PROB', replace = True)
#         b = df[['CREATIVE_NAME','PROB','REV/K']]
#         resamples_rev.append(pd.concat([a,b]))
#     sol_rev = []

#     #Sort all 1000 samples for CTR and UNIT/K and append to lists
#     for i in range(1000): 
#         resamples_ctr[i] = resamples_ctr[i].groupby('CREATIVE_NAME').agg({'CTR':'sum'})
#         resamples_ctr[i] = resamples_ctr[i].sort_values('CTR', ascending = False)
#         sol_ctr.append(list(resamples_ctr[i].index)) #list of 1000 lists having asset order

#         resamples_units[i] = resamples_units[i].groupby('CREATIVE_NAME').agg({'UNITS/K':'sum'})
#         resamples_units[i] = resamples_units[i].sort_values('UNITS/K', ascending = False)
#         sol_units.append(list(resamples_units[i].index))

#         #### NEW ####
#         resamples_rev[i] = resamples_rev[i].groupby('CREATIVE_NAME').agg({'REV/K':'sum'})
#         resamples_rev[i] = resamples_rev[i].sort_values('REV/K', ascending = False)
#         sol_rev.append(list(resamples_rev[i].index))

#     #Calculate sums for each asset and calculate combined metric sums
#     sum_ctr,sum_units,sum_rev,sum_result,asset_ordered,asset_ordered_rev = [],[],[],[],[],[]
#     unique_assets = df['CREATIVE_NAME'].unique()
#     for j in range(len(df['CREATIVE_NAME'].unique())):
#         sum_ctr.append(0)
#         sum_units.append(0)
#         #### NEW ####
#         sum_rev.append(0)
#         for i in range(1000):
#             sum_ctr[j] = sum_ctr[j] + sol_ctr[i].index(unique_assets[j])+1
#             sum_units[j] = sum_units[j] + sol_units[i].index(unique_assets[j])+1
#             #### NEW ####
#             sum_rev[j] = sum_rev[j] + sol_rev[i].index(unique_assets[j])+1
#         #Get a combined metric value by weighting ctr - 30% and units/k - 70%   
#         sum_result.append(np.round((sum_ctr[j]*0.3)+(sum_units[j]*0.7),1))

#     # print('Unsorted CTR:',sum_ctr)
#     # print('Unsorted UNIT/K:',sum_unitss)
#     # print("Unsorted combined:",sum_result)

#     #Dictionary to point sum values to asset names
#     pos_ctr = dict(zip(sum_ctr, df['CREATIVE_NAME'].unique()))
#     pos_units = dict(zip(sum_units, df['CREATIVE_NAME'].unique()))
#     #### NEW ####
#     pos_rev = dict(zip(sum_rev, df['CREATIVE_NAME'].unique()))
#     pos_combined = dict(zip(sum_result, df['CREATIVE_NAME'].unique()))

#     #Sort and print out re-ordered assets
#     sum_result.sort()
#     for i in sum_result:
#         asset_ordered.append(pos_combined[i])
#     # print('Re-ordered assets:',asset_ordered)

#     #### NEW ####
#     sum_rev.sort()
#     for i in sum_rev:
#         asset_ordered_rev.append(pos_rev[i])

#     # Get a list of avg ctr for all ordered assets
#     key_ctr = list(pos_ctr.keys())
#     val_ctr = list(pos_ctr.values())
#     avg_ctr = []
#     for i in asset_ordered:
#         position = val_ctr.index(i)
#         avg_ctr.append(np.round(key_ctr[position]/1000,2))

#     # Get a list of avg units/k for all ordered assets
#     key_units = list(pos_units.keys())
#     val_units = list(pos_units.values())
#     avg_units = []
#     for i in asset_ordered:
#         position = val_units.index(i)
#         avg_units.append(np.round(key_units[position]/1000,2))

#     #### NEW ####
#     key_rev = list(pos_rev.keys())
#     val_rev = list(pos_rev.values())
#     avg_rev = []
#     for i in asset_ordered_rev:
#         position = val_rev.index(i)
#         avg_rev.append(np.round(key_rev[position]/1000,2))

#     # Get a list of avg values for the combined metric for all ordered assets
#     avg_combined = [np.round((x / 1000),2) for x in sum_result]
#     #assets = pd.DataFrame(list(zip(asset_ordered,avg_ctr,avg_units,avg_combined)),columns= ['Re-ordered Assets','Avg CTR','Avg Units/K','Avg combined score'])
#     assets = pd.DataFrame(list(zip(asset_ordered,avg_combined,asset_ordered_rev,avg_rev)),columns= ['Re-ordered Assets CTR+UNITS/K','CTR+UNITS/K combined score','Re-ordered Assets REV/K','REV/K combined score'])

#     return assets

In [12]:
def asset_reordering(widget_name, start_date = '2022-04-07', end_date = '2022-04-21'):
    #Filter by widget, date range and calculate CTR, UNIT/K
    df = dataset[(dataset['WIDGET_NAME'] == widget_name)&(dataset['CMS_LEVEL'] == 'asset')]
    df['SESSIONS_DATE_EST'] = df['SESSIONS_DATE_EST'].astype(str)
    df = df[(df['SESSIONS_DATE_EST'] >= start_date)&(df['SESSIONS_DATE_EST'] <= end_date)]
    #print("Date Range: {} - {}".format(df['SESSIONS_DATE_EST'].min(),df['SESSIONS_DATE_EST'].max()))

    df['REV/K'] = ((df['ATTRIBUTED_REVENUE']/df['ASSET_SEGMENT_VIEWS'])*1000).round(2)
    df['REV/K'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df['REV/K'] = df['REV/K'].fillna(0)
    
    #Calculate mean impressions by assets
    asset_impressions = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean())
    asset_impressions = [x / df['ASSET_SEGMENT_VIEWS'].sum() for x in asset_impressions] #Calculate probability score of impressions by assets
    asset_names = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean().index)
    asset_dict = dict(zip(asset_names,asset_impressions))

    #Create probability column to sample assets by probability score
    df['PROB'] = df['CREATIVE_NAME'].map(asset_dict)

    df_rev = df[['CREATIVE_NAME','PROB','REV/K']]
    resamples_rev,sol_rev = [],[]
    for i in range(1000):
        a = df_rev.sample(n=5000, weights='PROB', replace = True)
        resamples_rev.append(pd.concat([a,df_rev]))

        resamples_rev[i] = resamples_rev[i].groupby('CREATIVE_NAME').agg({'REV/K':'sum'})
        resamples_rev[i] = resamples_rev[i].sort_values('REV/K', ascending = False)
        sol_rev.append(list(resamples_rev[i].index))

    #Calculate sums for each asset and calculate combined metric sums
    sum_rev,asset_ordered_rev = [],[]
    unique_assets = df['CREATIVE_NAME'].unique()
    for j in range(len(df['CREATIVE_NAME'].unique())):
        sum_rev.append(0)
        for i in range(1000):
            sum_rev[j] = sum_rev[j] + sol_rev[i].index(unique_assets[j])+1

    #Dictionary to point sum values to asset names
    pos_rev = dict(zip(sum_rev, df['CREATIVE_NAME'].unique()))

    #Sort and print out re-ordered assets
    sum_rev.sort()
    for i in sum_rev:
        asset_ordered_rev.append(pos_rev[i])

    # Get a list of avg ctr for all ordered assets
    key_rev = list(pos_rev.keys())
    val_rev = list(pos_rev.values())
    avg_rev = []
    for i in asset_ordered_rev:
        position = val_rev.index(i)
        avg_rev.append(np.round(key_rev[position]/1000,2))

    # Get a list of avg values for the combined metric for all ordered assets
    assets = pd.DataFrame(list(zip(asset_ordered_rev,avg_rev)),columns= ['Re-ordered Assets REV/K','REV/K combined score'])
    return assets

In [13]:
asset_reordering('todays-deals-body-30-grid')

,Re-ordered Assets REV/K,REV/K combined score
0,b2g1-solr-tile,1.01
1,jffd-solr-tile,2.22
2,deals-under-$10,2.78
3,spring-apparel-solr-tile,4.01
4,disney-solr-tile,5.20
5,goody-box-solr-tile,5.78
6,b3g4-solr-tile,7.00
7,dental-health-month-solr-tile,8.00
8,early-cyber-dog-buy-3-get-1-free,9.00
9,early-cyber-dog-treats,10.00


In [14]:
asset_reordering('todays-deals-body-40-sku-select-list')

,Re-ordered Assets REV/K,REV/K combined score
0,Kitten Toys & Treats,1.0
1,"Puppy Toys, Treats & Potty Training",2.0


In [15]:
asset_reordering('birthday-shop-body-20-grid')

,Re-ordered Assets REV/K,REV/K combined score
0,birthday-shop-all-dog-tile,1.0
1,birthday-dog-gift-sets-new,2.3
2,birthday-dog-cake-treats,2.7
3,birthday-dog-toys,4.0
4,birthday-dog-clothing,5.0
5,birthday-dog-homegoods,6.0


In [16]:
asset_reordering('chewy-picks-body-20-promo-list')

,Re-ordered Assets REV/K,REV/K combined score
0,chewy-picks-top-rated-spring,1.0
1,chewy-picks-new-arrivals-spring,2.0
2,chewy-picks-playtime-shop,3.0
3,chewy-picks-new-arrivals,4.0
4,chewy-picks-picks-of-the-month,5.0


In [44]:
#Tableau connections
# connection.deploy('Asset-reordering', asset_reordering, 'Re-ordered assets', override=True)
# connection.query('Asset-reordering','homepage-zone-10-body-10-promo-standard-nonsku-collection', '2022-02-10', '2022-02-17')['response']